# 6) Add BG dists and assignments to Modal Share data

## env

In [5]:
import os
import pandas as pd
import numpy as np

### Postgres connection

In [6]:
# Postgres
from dotenv import load_dotenv

load_dotenv()

from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy import text
# os.chdir('Documents/Grad School/Berkeley/Classes/CYP_204D - Multivariate Analysis in Planning/Project/204d_final_project/')
user = "postgres"
password = os.getenv('POSTGRES_PW')
host = "127.0.0.1"
port = "5432"
database = "gradschool"

pg_url = URL.create(
    drivername='postgresql',
    username='postgres',
    host='127.0.0.1',
    database='gradschool',
    password=os.getenv('POSTGRES_PW')
)

pg_engine = create_engine(pg_url)
# Test connection
try:
    with pg_engine.connect() as connection_str:
        print('Successfully connected to the PostgreSQL database')
except Exception as e:
    print(f'Sorry failed to connect: {e}')


Successfully connected to the PostgreSQL database


## Load Replica Data

In [4]:
# df_replica = {}
# for m in ['chi','phi','nyc','sfba']:
#     print(f'{m}..')
#     df_replica[m] = pd.read_sql(f'select * from cyp204d_final_project.{m}_replica_grpd;', con=pg_connection)
temp = pd.read_sql('select * from cyp204d_final_project.sfba_replica_grpd;', con=pg_engine.connect())

In [76]:
dfs_replica = {}
dfs_bg2zones = {}
for m in ['chi','phi','sfba','nyc']:
    print(f'{m}..')
    try:
        with pg_engine.connect() as pg_connection:
            #pd.read_sql_table('chi_replica_grpd', con=pg_connection, schema='cyp204d_final_project')
            dfs_replica[m] = pd.read_sql(f'select * from cyp204d_final_project.{m}_replica_grpd;', con=pg_connection)
        with pg_engine.connect() as pg_connection:
            dfs_bg2zones[m] = pd.read_sql(f'select * from cyp204d_final_project.{m}_bg2zones_od_dists;', con=pg_connection)
    except Exception as e:
        print(e)

chi..
phi..
sfba..
nyc..


In [78]:
cleaned_dfs = {}
for m in ['chi','nyc','phi','sfba']:
    print(f'{m}..')
    temp = dfs_replica[m].rename(columns={'O_bg_fips':'fips12_o', 'D_bg_fips':'fips12_d'}).copy()
    temp['trip_share'] = temp['n_trips'] / temp.groupby(['fips12_o','fips12_d'])['n_trips'].transform('sum')
    temp_pivot = temp[['fips12_o','fips12_d','primary_mode','n_trips','trip_share']].sort_values(by='n_trips',ascending=False).pivot(index=['fips12_o','fips12_d'], columns='primary_mode',values=['n_trips','trip_share'])
    df_clean = temp_pivot['trip_share'].copy()
    df_clean['total_trips'] = temp_pivot['n_trips'].sum(axis=1)
    df_clean = df_clean.fillna(0)[['total_trips'] + df_clean.columns.to_list()[0:-1]]
    cleaned_dfs[m] = df_clean.copy()


chi..
nyc..
phi..
sfba..


In [98]:
# # dfs_bg2zones['chi'].set_index(['fips12_o','fips12_d']).drop('index',axis=1)
final_dataset = {}
for m in ['chi','phi','nyc','sfba']:
    print(f'{m}..')
    final_dataset[m] = (
        dfs_bg2zones[m]
        .set_index(['fips12_o','fips12_d'])
        .drop('index',axis=1)
        .merge(
            cleaned_dfs[m],
            left_index=True,
            right_index=True
        )
    )

# #dfs_bg2zones['chi']['fips12_o']
# #cleaned_dfs['nyc']
# dfs_replica['nyc']
#dfs_replica['sfba']

chi..
phi..
nyc..
sfba..


In [100]:
# (
#     pd.concat({k: x.groupby('zone_same')[['total_trips','biking','walking','on_demand_auto','private_auto','auto_passenger','public_transit']].sum() for k, x in final_dataset.items()})
for m in ['chi','phi','nyc','sfba']:
    print(f'{m}..')
    final_dataset[m].to_sql(f'{m}_final_datasets', con = pg_engine.connect(), schema='cyp204d_final_project', if_exists='replace')

chi..
phi..
nyc..
sfba..


In [172]:

for m in final_dataset.keys():
    print(f"'{m}', {final_dataset[m]['zoneID_o'].unique().shape[0]}, {final_dataset[m]['zoneID_d'].unique().shape[0]}")

'chi', 54, 54
'phi', 15, 15
'nyc', 89, 88
'sfba', 63, 63


## Mapping

In [137]:
import pickle
import geopandas as gpd
zones = {}
for m in ['chi','phi','nyc','sfba']:
    with open(f'../data/temp/zones_{m}', 'rb') as fp:
        zones[m] = gpd.GeoDataFrame(pickle.load(fp).reset_index().drop('index',axis=1).rename(columns={0:'geometry'}))



In [165]:
#!pip install folium
#!pip install mapclassify
# zones['chi'].reset_index().assign(temp=lambda x: x.index % 10).explore('temp')
#zones['phi'].reset_index().assign(temp=lambda x: x.index % 10).explore('temp')
# zones['nyc'].reset_index().assign(temp=lambda x: x.index % 10).explore('temp')
# zones['sfba'].reset_index().assign(temp=lambda x: x.index % 10).explore('temp')
# #zones['chi'].reset_index().assign(temp=lambda x: x.index % 10).explore('temp')

#pd.concat([zones['chi'], zones['phi'], zones['sfba'], zones['chi']]).reset_index().assign(temp=lambda x: x.index % 10).explore('temp')
for m in zones.keys():
   print(f"{m}..")
   print(f"\t{zones[m].shape[0]}")
   print(f"\t{zones[m][zones[m].area > 1000000].shape[0]}")

#zones['chi'][zones['chi'].area > 1000000]

chi..
	1994
	54
phi..
	291
	15
nyc..
	1683
	79
sfba..
	3573
	70


In [ ]:
pd.concat(zones)

geometry
chi  0     POLYGON ((-9825463.572 5171001.965, -9825427.4...
     1     POLYGON ((-9825463.229 5170957.244, -9825452.1...
     2     POLYGON ((-9825461.543 5170868.713, -9825460.9...
     3     POLYGON ((-9825435.964 5167665.649, -9825436.8...
     4     POLYGON ((-9825399.597 5161363.831, -9825399.3...
...                                                      ...
sfba 3568  POLYGON ((-13528286.059 4450513.835, -13528285...
     3569  POLYGON ((-13533593.818 4545222.571, -13533769...
     3570  POLYGON ((-13533796.24 4545471.103, -13533775....
     3571  POLYGON ((-13532807.972 4546081.22, -13532807....
     3572  POLYGON ((-13533104.002 4545934.351, -13533122...

[7541 rows x 1 columns]